In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/checkpoint-for-translation/translation_checkpoint-loaded.csv
/kaggle/input/translation-checkpoint2/translation2_checkpoint-loaded.csv
/kaggle/input/tedy-with-lang/tedx_videos_extended_with_lang.csv


In [4]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

article_hi = "Empresas: um espetáculo à parte"
article_ar = "Errei, parei! Que erro!"

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

# translate Hindi to French
tokenizer.src_lang = "pt_XX"
encoded_hi = tokenizer(article_hi, return_tensors="pt")
generated_tokens = model.generate(
    **encoded_hi,
    forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
)
tekst1 = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
# => "Le chef de l 'ONU affirme qu 'il n 'y a pas de solution militaire dans la Syrie."

# translate Arabic to English
tokenizer.src_lang = "it_IT"
encoded_ar = tokenizer(article_ar, return_tensors="pt")
generated_tokens = model.generate(
    **encoded_ar,
    forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
)
tekst2 = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
# => "The Secretary-General of the United Nations says there is no military solution in Syria."


In [5]:
print(tekst1)
print(tekst2)

['Companies: a one-off show.']
["You're wrong, stop! You're wrong!"]


In [6]:
import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

# Sprawdzenie, czy GPU jest dostępne
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

article_hi = "Empresas: um espetáculo à parte"
article_ar = "Errei, parei! Que erro!"

# Załadowanie modelu i tokenizera
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt").to(device)
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

# Tłumaczenie z Hindi na Francuski
tokenizer.src_lang = "pt_XX"
encoded_hi = tokenizer(article_hi, return_tensors="pt").to(device)
generated_tokens = model.generate(
    **encoded_hi,
    forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
)
tekst1 = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
print(tekst1)  # => "Le chef de l 'ONU affirme qu 'il n 'y a pas de solution militaire dans la Syrie."

# Tłumaczenie z Arabskiego na Angielski
tokenizer.src_lang = "it_IT"
encoded_ar = tokenizer(article_ar, return_tensors="pt").to(device)
generated_tokens = model.generate(
    **encoded_ar,
    forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
)
tekst2 = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
print(tekst2)  # => "The Secretary-General of the United Nations says there is no military solution in Syria."


['Companies: a one-off show.']
["You're wrong, stop! You're wrong!"]


In [7]:
print(tekst1)
print(tekst2)

['Companies: a one-off show.']
["You're wrong, stop! You're wrong!"]


In [3]:
language_dict = {
    "ar": "ar_AR",
    "cs": "cs_CZ",
    "de": "de_DE",
    "en": "en_XX",
    "es": "es_XX",
    "et": "et_EE",
    "fi": "fi_FI",
    "fr": "fr_XX",
    "gu": "gu_IN",
    "hi": "hi_IN",
    "it": "it_IT",
    "ja": "ja_XX",
    "kk": "kk_KZ",
    "ko": "ko_KR",
    "lt": "lt_LT",
    "lv": "lv_LV",
    "my": "my_MM",
    "ne": "ne_NP",
    "nl": "nl_XX",
    "ro": "ro_RO",
    "ru": "ru_RU",
    "si": "si_LK",
    "tr": "tr_TR",
    "vi": "vi_VN",
    "zh": "zh_CN",
    "af": "af_ZA",
    "az": "az_AZ",
    "bn": "bn_IN",
    "fa": "fa_IR",
    "he": "he_IL",
    "hr": "hr_HR",
    "id": "id_ID",
    "ka": "ka_GE",
    "km": "km_KH",
    "mk": "mk_MK",
    "ml": "ml_IN",
    "mn": "mn_MN",
    "mr": "mr_IN",
    "pl": "pl_PL",
    "ps": "ps_AF",
    "pt": "pt_XX",
    "sv": "sv_SE",
    "sw": "sw_KE",
    "ta": "ta_IN",
    "te": "te_IN",
    "th": "th_TH",
    "tl": "tl_XX",
    "uk": "uk_UA",
    "ur": "ur_PK",
    "xh": "xh_ZA",
    "gl": "gl_ES",
    "sl": "sl_SI"
}

In [ ]:
# translation

In [24]:
from tqdm import tqdm
def translate_text(df):
    translated_texts = []  # Lista na przetłumaczone teksty
    
    # Dodajemy pasek postępu do pętli
    for _, row in tqdm(df.iterrows(), total=df.shape[0], desc="Tłumaczenie tekstów", unit="wiersz"):
        # Sprawdzamy, czy język jest w słowniku
        language_code = row['language']  # Zakłada, że język w 'language' to np. "pt_XX"
        
        if language_code in language_dict:
            # Ustawiamy język źródłowy tokenizera
            tokenizer.src_lang = language_dict[language_code]
            
            # Przetwarzamy tekst
            encoded_text = tokenizer(row['title'], return_tensors="pt").to(device)
            generated_tokens = model.generate(
                **encoded_text,
                forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]  # Tłumaczymy na angielski
            )
            translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
            translated_texts.append(translated_text)
        else:
            # Jeśli język nie jest w słowniku, dodajemy None
            translated_texts.append(None)
    
    # Dodajemy kolumnę z przetłumaczonymi tekstami
    df['translated_text'] = translated_texts
    return df

In [3]:
import pandas as pd
df_en = pd.read_csv("/kaggle/input/tedy-with-lang/tedx_videos_extended_with_lang.csv")
df_en

,full_title,views,date_str,date,year,title,speaker,event,language
0,What Shakespeare teaches us about modern consp...,15518.0,23 godziny temu 13 minut i 11 sekund,2024-12-24 01:00:00,2024,What Shakespeare teaches us about modern consp...,Dr. Paul Budra,TEDxSurreySalon,en
1,How poetry saved me from a cult | Diannely Ant...,14758.0,1 dzień temu 21 minut,2024-12-24 00:00:00,2024,How poetry saved me from a cult,Diannely Antigua,TEDxPortsmouth,en
2,Why language shapes identity (more than race) ...,25684.0,2 dni temu 13 minut i 52 sekundy,2024-12-23 00:00:00,2024,Why language shapes identity (more than race),Malaka Grant,TEDxGeorge,en
3,On designing a presidential library | Craig Dy...,14181.0,3 dni temu 20 minut,2024-12-22 00:00:00,2024,On designing a presidential library,Craig Dykers,TEDxFargo,es
4,Why chasing happiness is nuts: What to do inst...,10858.0,4 dni temu 16 minut,2024-12-21 00:00:00,2024,Why chasing happiness is nuts: What to do instead,Lenorë Lambert,TEDxBillings,en
...,...,...,...,...,...,...,...,...,...
226747,TEDxWarwick - Professor Vinesh Raja - 2/28/09,2474.0,15 years ago 29 minutes,2009-12-25,2009,TEDxWarwick - Professor Vinesh Raja - 2/28/09,NaN,NaN,en
226748,TEDxWarwick - Professor Steve Fuller - 2/28/09,8460.0,15 years ago 24 minutes,2009-12-25,2009,TEDxWarwick - Professor Steve Fuller - 2/28/09,NaN,NaN,en
226749,TEDxWarwick - Francois Grey - 2/28/09,3480.0,15 years ago 27 minutes,2009-12-25,2009,TEDxWarwick - Francois Grey - 2/28/09,NaN,NaN,en
226750,TEDxWarwick - Professor Andrew Oswald - 2/28/09,6390.0,15 years ago 24 minutes,2009-12-25,2009,TEDxWarwick - Professor Andrew Oswald - 2/28/09,NaN,NaN,en


In [4]:
df = df_en[df_en['language'] != 'en']
df

,full_title,views,date_str,date,year,title,speaker,event,language
3,On designing a presidential library | Craig Dy...,14181.0,3 dni temu 20 minut,2024-12-22 00:00:00,2024,On designing a presidential library,Craig Dykers,TEDxFargo,es
22,Empresas: um espetáculo à parte | Flávia Macie...,118.0,5 dni temu 6 minut i 52 sekundy,2024-12-20 00:00:00,2024,Empresas: um espetáculo à parte,Flávia Maciel,TEDxMaringáStudio,pt
23,O poder das palavras | Nicolas Oliveira | TEDx...,139.0,5 dni temu 6 minut i 17 sekund,2024-12-20 00:00:00,2024,O poder das palavras,Nicolas Oliveira,TEDxMaringáStudio,pt
24,"Errei, parei! Que erro! | Fábio Paixão | TEDxM...",58.0,5 dni temu 8 minut i 14 sekund,2024-12-20 00:00:00,2024,"Errei, parei! Que erro!",Fábio Paixão,TEDxMaringáStudio,it
25,KENDİNİZE HOŞGELDİNİZ | Hakan Bilgin | TEDxAta...,504.0,5 dni temu 16 minut,2024-12-20 00:00:00,2024,KENDİNİZE HOŞGELDİNİZ,Hakan Bilgin,TEDxAtapark,vi
...,...,...,...,...,...,...,...,...,...
226741,TEDxKL - Yasmin Ahmad - 6/3/09,179759.0,"15 years ago 14 minutes, 35 seconds",2009-12-25,2009,TEDxKL - Yasmin Ahmad - 6/3/09,NaN,NaN,tr
226743,TEDxWarwick | Solitaire Townsend,4353.0,15 years ago 32 minutes,2009-12-25,2009,TEDxWarwick,Solitaire Townsend,NaN,pl
226744,TEDxWarwick - Professor Steve Furber - 2/28/09,4291.0,15 years ago 24 minutes,2009-12-25,2009,TEDxWarwick - Professor Steve Furber - 2/28/09,NaN,NaN,de
226745,TEDxWarwick | Dr Ian Pearson,12286.0,15 years ago 25 minutes,2009-12-25,2009,TEDxWarwick,Dr Ian Pearson,NaN,pl


In [25]:
# Przetłumaczenie tekstów
df_translated = translate_text(df)

# Wyświetlanie przetłumaczonego DataFrame
print(df_translated)

Tłumaczenie tekstów:   0%|          | 136/70213 [01:13<10:28:39,  1.86wiersz/s]


KeyboardInterrupt: 

In [26]:
#optimized func
import pandas as pd
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import torch
from tqdm import tqdm

# Sprawdzenie, czy GPU jest dostępne
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Załadowanie modelu i tokenizera raz
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt").to(device)
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

# Słownik z językami (przykładowy fragment)
language_dict = {
    "ar": "ar_AR",
    "cs": "cs_CZ",
    "de": "de_DE",
    "en": "en_XX",
    "es": "es_XX",
    "fr": "fr_XX",
    "hi": "hi_IN",
    "it": "it_IT",
    "pl": "pl_PL",
    "pt": "pt_XX",
    "ru": "ru_RU",
    "zh": "zh_CN"
}

def translate_text(df):
    translated_texts = []  # Lista na przetłumaczone teksty
    
    # Będziemy tłumaczyć partie tekstów, zamiast pojedynczych wierszy
    batch_size = 16  # Możesz dostosować rozmiar partii
    for i in tqdm(range(0, df.shape[0], batch_size), desc="Tłumaczenie tekstów", unit="partia"):
        batch = df.iloc[i:i+batch_size]  # Pobieramy partię wierszy
        
        # Lista dla przetłumaczonych tekstów w partii
        batch_translated_texts = []
        
        for _, row in batch.iterrows():
            # Sprawdzamy, czy język jest w słowniku
            language_code = row['language']  # Zakłada, że język w 'language' to np. "pt_XX"
            
            if language_code in language_dict:
                # Ustawiamy język źródłowy tokenizera
                tokenizer.src_lang = language_dict[language_code]
                
                # Przetwarzamy tekst
                encoded_text = tokenizer(row['title'], return_tensors="pt").to(device)
                generated_tokens = model.generate(
                    **encoded_text,
                    forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]  # Tłumaczymy na angielski
                )
                translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
                batch_translated_texts.append(translated_text)
            else:
                # Jeśli język nie jest w słowniku, dodajemy None
                batch_translated_texts.append(None)
        
        # Dodajemy przetłumaczone teksty do listy
        translated_texts.extend(batch_translated_texts)
    
    # Dodajemy kolumnę z przetłumaczonymi tekstami
    df['translated_text'] = translated_texts
    return df


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [27]:
# Przetłumaczenie tekstów
df_translated = translate_text(df)

Tłumaczenie tekstów:   0%|          | 12/4389 [01:30<9:12:03,  7.57s/partia] 


KeyboardInterrupt: 

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
language_dict = {
    "ar": "ar_AR",
    "cs": "cs_CZ",
    "de": "de_DE",
    "en": "en_XX",
    "es": "es_XX",
    "et": "et_EE",
    "fi": "fi_FI",
    "fr": "fr_XX",
    "gu": "gu_IN",
    "hi": "hi_IN",
    "it": "it_IT",
    "ja": "ja_XX",
    "kk": "kk_KZ",
    "ko": "ko_KR",
    "lt": "lt_LT",
    "lv": "lv_LV",
    "my": "my_MM",
    "ne": "ne_NP",
    "nl": "nl_XX",
    "ro": "ro_RO",
    "ru": "ru_RU",
    "si": "si_LK",
    "tr": "tr_TR",
    "vi": "vi_VN",
    "zh": "zh_CN",
    "af": "af_ZA",
    "az": "az_AZ",
    "bn": "bn_IN",
    "fa": "fa_IR",
    "he": "he_IL",
    "hr": "hr_HR",
    "id": "id_ID",
    "ka": "ka_GE",
    "km": "km_KH",
    "mk": "mk_MK",
    "ml": "ml_IN",
    "mn": "mn_MN",
    "mr": "mr_IN",
    "pl": "pl_PL",
    "ps": "ps_AF",
    "pt": "pt_XX",
    "sv": "sv_SE",
    "sw": "sw_KE",
    "ta": "ta_IN",
    "te": "te_IN",
    "th": "th_TH",
    "tl": "tl_XX",
    "uk": "uk_UA",
    "ur": "ur_PK",
    "xh": "xh_ZA",
    "gl": "gl_ES",
    "sl": "sl_SI"
}

In [2]:
import pandas as pd
df_en = pd.read_csv("/kaggle/input/tedy-with-lang/tedx_videos_extended_with_lang.csv")
df_en

,full_title,views,date_str,date,year,title,speaker,event,language
0,What Shakespeare teaches us about modern consp...,15518.0,23 godziny temu 13 minut i 11 sekund,2024-12-24 01:00:00,2024,What Shakespeare teaches us about modern consp...,Dr. Paul Budra,TEDxSurreySalon,en
1,How poetry saved me from a cult | Diannely Ant...,14758.0,1 dzień temu 21 minut,2024-12-24 00:00:00,2024,How poetry saved me from a cult,Diannely Antigua,TEDxPortsmouth,en
2,Why language shapes identity (more than race) ...,25684.0,2 dni temu 13 minut i 52 sekundy,2024-12-23 00:00:00,2024,Why language shapes identity (more than race),Malaka Grant,TEDxGeorge,en
3,On designing a presidential library | Craig Dy...,14181.0,3 dni temu 20 minut,2024-12-22 00:00:00,2024,On designing a presidential library,Craig Dykers,TEDxFargo,es
4,Why chasing happiness is nuts: What to do inst...,10858.0,4 dni temu 16 minut,2024-12-21 00:00:00,2024,Why chasing happiness is nuts: What to do instead,Lenorë Lambert,TEDxBillings,en
...,...,...,...,...,...,...,...,...,...
226747,TEDxWarwick - Professor Vinesh Raja - 2/28/09,2474.0,15 years ago 29 minutes,2009-12-25,2009,TEDxWarwick - Professor Vinesh Raja - 2/28/09,NaN,NaN,en
226748,TEDxWarwick - Professor Steve Fuller - 2/28/09,8460.0,15 years ago 24 minutes,2009-12-25,2009,TEDxWarwick - Professor Steve Fuller - 2/28/09,NaN,NaN,en
226749,TEDxWarwick - Francois Grey - 2/28/09,3480.0,15 years ago 27 minutes,2009-12-25,2009,TEDxWarwick - Francois Grey - 2/28/09,NaN,NaN,en
226750,TEDxWarwick - Professor Andrew Oswald - 2/28/09,6390.0,15 years ago 24 minutes,2009-12-25,2009,TEDxWarwick - Professor Andrew Oswald - 2/28/09,NaN,NaN,en


In [4]:
df = df_en[df_en['language'] != 'en']
df

,full_title,views,date_str,date,year,title,speaker,event,language
3,On designing a presidential library | Craig Dy...,14181.0,3 dni temu 20 minut,2024-12-22 00:00:00,2024,On designing a presidential library,Craig Dykers,TEDxFargo,es
22,Empresas: um espetáculo à parte | Flávia Macie...,118.0,5 dni temu 6 minut i 52 sekundy,2024-12-20 00:00:00,2024,Empresas: um espetáculo à parte,Flávia Maciel,TEDxMaringáStudio,pt
23,O poder das palavras | Nicolas Oliveira | TEDx...,139.0,5 dni temu 6 minut i 17 sekund,2024-12-20 00:00:00,2024,O poder das palavras,Nicolas Oliveira,TEDxMaringáStudio,pt
24,"Errei, parei! Que erro! | Fábio Paixão | TEDxM...",58.0,5 dni temu 8 minut i 14 sekund,2024-12-20 00:00:00,2024,"Errei, parei! Que erro!",Fábio Paixão,TEDxMaringáStudio,it
25,KENDİNİZE HOŞGELDİNİZ | Hakan Bilgin | TEDxAta...,504.0,5 dni temu 16 minut,2024-12-20 00:00:00,2024,KENDİNİZE HOŞGELDİNİZ,Hakan Bilgin,TEDxAtapark,vi
...,...,...,...,...,...,...,...,...,...
226741,TEDxKL - Yasmin Ahmad - 6/3/09,179759.0,"15 years ago 14 minutes, 35 seconds",2009-12-25,2009,TEDxKL - Yasmin Ahmad - 6/3/09,NaN,NaN,tr
226743,TEDxWarwick | Solitaire Townsend,4353.0,15 years ago 32 minutes,2009-12-25,2009,TEDxWarwick,Solitaire Townsend,NaN,pl
226744,TEDxWarwick - Professor Steve Furber - 2/28/09,4291.0,15 years ago 24 minutes,2009-12-25,2009,TEDxWarwick - Professor Steve Furber - 2/28/09,NaN,NaN,de
226745,TEDxWarwick | Dr Ian Pearson,12286.0,15 years ago 25 minutes,2009-12-25,2009,TEDxWarwick,Dr Ian Pearson,NaN,pl


In [11]:
# Przetłumaczenie tekstów
df_translated = translate_text(df)

Tłumaczenie tekstów:  84%|████████▍ | 43/51 [2:22:34<26:31, 198.94s/język]    


TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [13]:
df_translated

NameError: name 'df_translated' is not defined

In [14]:
# FINAL AFTER ERROR

In [4]:
# optimized2
import pandas as pd
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import torch
from tqdm import tqdm

# Sprawdzenie, czy GPU jest dostępne
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Załadowanie modelu i tokenizera raz
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt").to(device)
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

In [10]:
def translate_text(df, batch_size=8, checkpoint_file="translation_checkpoint.csv"):
    translated_texts = [None] * len(df)  # Lista na przetłumaczone teksty
    language_counter = 0  # Licznik do zapisywania checkpointu co 5 języków

    # Zapisz oryginalne indeksy
    df = df.copy()  # Upewniamy się, że pracujemy na kopii DataFrame
    df.loc[:, 'original_index'] = df.index

    # Sortowanie danych według języka
    df_sorted = df.sort_values(by='language').reset_index(drop=True)

    # Przetwarzanie tekstów po kolei
    current_language = None
    for idx, row in tqdm(df_sorted.iterrows(), desc="Tłumaczenie tekstów", unit="wiersz"):
        language_code = row['language']
        if language_code != current_language:
            current_language = language_code
            if language_code in language_dict:
                print(f"Przetwarzanie języka: {language_code}")
            else:
                print(f"Język {language_code} nie jest obsługiwany")

        if language_code in language_dict:
            # Ustawiamy język źródłowy tokenizera
            tokenizer.src_lang = language_dict[language_code]

            # Walidacja danych: odrzucanie nieprawidłowych tekstów
            text = row['title']
            if isinstance(text, str) and text.strip() != "":
                # Tokenizacja
                encoded_texts = tokenizer([text], padding=True, truncation=True, return_tensors="pt").to(device)
                
                # Generowanie tłumaczeń
                generated_tokens = model.generate(
                    **encoded_texts,
                    forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]  # Tłumaczymy na angielski
                )
                
                # Dekodowanie przetłumaczonego tekstu
                translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
                translated_texts[idx] = translated_text
            else:
                translated_texts[idx] = None

            language_counter += 1
            if language_counter % 5000 == 0:
                # Zapisujemy checkpoint co 5 języków
                checkpoint_df = df_sorted.copy()
                checkpoint_df['translated_text'] = translated_texts
                checkpoint_df.to_csv(checkpoint_file, index=False)
        else:
            translated_texts[idx] = None

    # Przypisanie przetłumaczonych tekstów do oryginalnego DataFrame
    df_sorted.loc[:, 'translated_text'] = translated_texts

    # Przywrócenie oryginalnego porządku
    result_df = df_sorted.sort_values(by='original_index').drop(columns=['original_index']).reset_index(drop=True)

    # Zapisz końcowy wynik do pliku .csv po przetłumaczeniu wszystkich tekstów
    result_df.to_csv(checkpoint_file, index=False)
    
    return result_df

In [11]:
df_translated = translate_text(df, batch_size=16, checkpoint_file="/kaggle/working/translation_checkpoint.csv")

Tłumaczenie tekstów: 1wiersz [00:00,  4.52wiersz/s]

Przetwarzanie języka: af


Tłumaczenie tekstów: 957wiersz [02:22,  7.07wiersz/s]

Przetwarzanie języka: ar


Tłumaczenie tekstów: 1795wiersz [04:57,  4.26wiersz/s]

...

Język cy nie jest obsługiwany
Język da nie jest obsługiwany
Przetwarzanie języka: de


Tłumaczenie tekstów: 10124wiersz [20:22,  4.49wiersz/s] 

Język el nie jest obsługiwany
Przetwarzanie języka: es


Tłumaczenie tekstów: 19631wiersz [2:56:45,  1.85wiersz/s]


KeyboardInterrupt: 

In [12]:
df

,text,language,translated_text
0,Empresas: um espetáculo à parte,pt_XX,None
1,"Errei, parei! Que erro!",pt_XX,None
2,None,pt_XX,None
3,Un texte en français.,fr_XX,None
4,Texto adicional em português.,pt_XX,None
5,Otro texto en español.,es_XX,None


# AFTER CHECKPOINT

In [6]:
import pandas as pd
df = pd.read_csv("/kaggle/input/checkpoint-for-translation/translation_checkpoint-loaded.csv")

In [15]:
df_translated = translate_text(df, batch_size=64, checkpoint_file="/kaggle/working/translation_checkpoint.csv")

Tłumaczenie tekstów: 2412wiersz [00:00, 13816.73wiersz/s]

Przetwarzanie języka: af
Język bg nie jest obsługiwany
Język ca nie jest obsługiwany
Język cy nie jest obsługiwany
Język da nie jest obsługiwany
Język el nie jest obsługiwany
Przetwarzanie języka: es


Tłumaczenie tekstów: 4135wiersz [01:05, 62.67wiersz/s]   


KeyboardInterrupt: 

In [14]:
df

,Unnamed: 0,full_title,views,date_str,date,year,title,speaker,event,language,original_index,translated_text
0,738,"Stories told, stories heard, stories kept | Be...",345.0,6 years ago 16 minutes,2018-12-25,2018,"Stories told, stories heard, stories kept",Ben McCorkle,TEDxYearlingRoad,af,103496,NaN
1,1795,Да върнем смисъла на думите | Maria Laleva | T...,2213.0,"1 year ago 13 minutes, 31 seconds",2023-12-25,2023,Да върнем смисъла на думите,Maria Laleva,TEDxVitoshaSalon,bg,33376,NaN
2,1796,5 хвилин щоб врятувати життя | Галина Алмазова...,1810.0,7 years ago 21 minutes,2017-12-25,2017,5 хвилин щоб врятувати життя,Галина Алмазова,TEDxKyiv,bg,139536,NaN
3,1797,Как една непобедима болест те учи да побеждава...,2655.0,"1 year ago 13 minutes, 19 seconds",2023-12-25,2023,Как една непобедима болест те учи да побеждаваш?,Victoria Radoslavova,TEDxVitoshaSalon,bg,33374,NaN
4,1798,Жити! Творити! Дивуватися! | Олена Загрєбіна (...,283.0,5 years ago 16 minutes,2019-12-25,2019,Жити! Творити! Дивуватися!,Олена Загрєбіна (Mura Ura),Mura Ura (Olena Zagrebina) | TEDxChernihiv,bg,82129,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
55209,70208,「網紅經濟」是怎麼一回事？ | 育綾 田 | TEDxDongWuU,1746.0,4 years ago 22 minutes,2020-12-25,2020,「網紅經濟」是怎麼一回事？,育綾 田,TEDxDongWuU,zh-tw,67023,NaN
55210,70209,「我跟你一樣都是人」，知道「根」在哪裡，我們就不會徬徨 | 知學 田 | TEDxDongWuU,16046.0,4 years ago 22 minutes,2020-12-25,2020,「我跟你一樣都是人」，知道「根」在哪裡，我們就不會徬徨,知學 田,TEDxDongWuU,zh-tw,67024,NaN
55211,70210,「法律」猶如阿拉丁神燈，試著了解它，它就會保護你！ | 宇安 陳 | TEDxDongWuU,1956.0,4 years ago 17 minutes,2020-12-25,2020,「法律」猶如阿拉丁神燈，試著了解它，它就會保護你！,宇安 陳,TEDxDongWuU,zh-tw,67025,NaN
55212,70211,不管你移動到哪裡，你都是人 | 江 彥杰 | TEDxChinaMedicalU,564.0,7 years ago 19 minutes,2017-12-25,2017,不管你移動到哪裡，你都是人,江 彥杰,TEDxChinaMedicalU,zh-tw,131891,NaN


# Added batching

In [5]:
import pandas as pd
from tqdm import tqdm

def translate_text(df, batch_size=8, checkpoint_file="translation_checkpoint.csv"):
    translated_texts = [None] * len(df)  # Lista na przetłumaczone teksty
    language_counter = 0  # Licznik do zapisywania checkpointu co 5000 rekordów

    # Zapisz oryginalne indeksy
    df = df.copy()  # Upewniamy się, że pracujemy na kopii DataFrame
    df.loc[:, 'original_index'] = df.index

    # Sortowanie danych według języka
    df_sorted = df.sort_values(by='language').reset_index(drop=True)

    # Przetwarzanie tekstów w batchach
    current_language = None
    batch_texts = []
    batch_indices = []

    for idx, row in tqdm(df_sorted.iterrows(), desc="Tłumaczenie tekstów", unit="wiersz"):
        language_code = row['language']
        if language_code != current_language:
            if batch_texts:
                # Przetwarzanie batcha
                encoded_texts = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt").to(device)
                generated_tokens = model.generate(
                    **encoded_texts,
                    forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
                )
                translated_batch = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
                for i, translated_text in enumerate(translated_batch):
                    translated_texts[batch_indices[i]] = translated_text

                batch_texts = []
                batch_indices = []

            current_language = language_code
            if language_code in language_dict:
                print(f"Przetwarzanie języka: {language_code}")
            else:
                print(f"Język {language_code} nie jest obsługiwany")

        if language_code in language_dict:
            tokenizer.src_lang = language_dict[language_code]
            text = row['title']
            if isinstance(text, str) and text.strip() != "":
                batch_texts.append(text)
                batch_indices.append(idx)

                if len(batch_texts) == batch_size:
                    encoded_texts = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt").to(device)
                    generated_tokens = model.generate(
                        **encoded_texts,
                        forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
                    )
                    translated_batch = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
                    for i, translated_text in enumerate(translated_batch):
                        translated_texts[batch_indices[i]] = translated_text

                    batch_texts = []
                    batch_indices = []

            else:
                translated_texts[idx] = None

            language_counter += 1
            if language_counter % 5000 == 0:
                checkpoint_df = df_sorted.copy()
                checkpoint_df['translated_text'] = translated_texts
                checkpoint_df.to_csv(checkpoint_file, index=False)
        else:
            translated_texts[idx] = None

    if batch_texts:
        encoded_texts = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt").to(device)
        generated_tokens = model.generate(
            **encoded_texts,
            forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
        )
        translated_batch = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        for i, translated_text in enumerate(translated_batch):
            translated_texts[batch_indices[i]] = translated_text

    df_sorted.loc[:, 'translated_text'] = translated_texts
    result_df = df_sorted.sort_values(by='original_index').drop(columns=['original_index']).reset_index(drop=True)
    result_df.to_csv(checkpoint_file, index=False)
    
    return result_df

In [7]:
df_translated = translate_text(df, batch_size=32, checkpoint_file="/kaggle/working/translation2_checkpoint.csv")

Tłumaczenie tekstów: 0wiersz [00:00, ?wiersz/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Przetwarzanie języka: af


Tłumaczenie tekstów: 2435wiersz [00:01, 3023.06wiersz/s]

Język bg nie jest obsługiwany
Język ca nie jest obsługiwany
Język cy nie jest obsługiwany
Język da nie jest obsługiwany
Język el nie jest obsługiwany
Przetwarzanie języka: es


Tłumaczenie tekstów: 9794wiersz [51:49,  1.83wiersz/s]  

...

Język no nie jest obsługiwany
Przetwarzanie języka: pl


Tłumaczenie tekstów: 35101wiersz [1:12:16, 17.64wiersz/s] 

Przetwarzanie języka: pt


Tłumaczenie tekstów: 38619wiersz [1:15:24,  8.54wiersz/s]


KeyboardInterrupt: 

In [6]:
import pandas as pd
df = pd.read_csv("/kaggle/input/translation-checkpoint2/translation2_checkpoint-loaded.csv")

In [7]:
df

,Unnamed: 0.1,Unnamed: 0,full_title,views,date_str,date,year,title,speaker,event,language,original_index,translated_text
0,0,738,"Stories told, stories heard, stories kept | Be...",345.0,6 years ago 16 minutes,2018-12-25,2018,"Stories told, stories heard, stories kept",Ben McCorkle,TEDxYearlingRoad,af,0,NaN
1,1,2005,Звездната система и следващото поколение в шоу...,6711.0,2 years ago 22 minutes,2022-12-25,2022,Звездната система и следващото поколение в шоу...,Магърдич Халваджиян,TEDxMladost,bg,211,NaN
2,2,2004,Защо не?възможната промяна в образованието | D...,874.0,"6 years ago 13 minutes, 1 second",2018-12-25,2018,Защо не?възможната промяна в образованието,Dimitrina Dokimova,TEDxPrimorskiPark,bg,210,NaN
3,3,2003,Родители от следващо поколение | Ася Демирева ...,4382.0,2 years ago 17 minutes,2022-12-25,2022,Родители от следващо поколение,Ася Демирева,TEDxMladost,bg,209,NaN
4,4,2002,Не-обещание и не-изпълнение | Еленко Еленков |...,7935.0,10 years ago 17 minutes,2014-12-25,2014,Не-обещание и не-изпълнение,Еленко Еленков,TEDxBG,bg,208,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25219,25219,69960,人人都可以是大藝術家 | 謝昕璇 | Hsin-Hsuan Hsieh | TEDxNDHU,1550.0,7 years ago 17 minutes,2017-12-25,2017,人人都可以是大藝術家,謝昕璇,Hsin-Hsuan Hsieh | TEDxNDHU,zh-tw,54961,NaN
25220,25220,69959,自信才是最好的時尚單品 | 圈 入準 | TEDxChungChengU,1037.0,2 years ago 19 minutes,2022-12-25,2022,自信才是最好的時尚單品,圈 入準,TEDxChungChengU,zh-tw,54960,NaN
25221,25221,69958,「嘿！多久沒到書店走走啦？」 | 王 政中 | TEDxDongWuU,124.0,7 months ago 21 minutes,2024-05-25 00:00:00,2024,「嘿！多久沒到書店走走啦？」,王 政中,TEDxDongWuU,zh-tw,54959,NaN
25222,25222,69967,那些年。孩子與動物教我的事 | 呂 軍逸 | TEDxProvidenceUniversity,754.0,3 years ago 24 minutes,2021-12-25,2021,那些年。孩子與動物教我的事,呂 軍逸,TEDxProvidenceUniversity,zh-tw,54968,NaN


In [8]:
df_translated = translate_text(df, batch_size=24, checkpoint_file="/kaggle/working/translation3_checkpoint.csv")

Tłumaczenie tekstów: 0wiersz [00:00, ?wiersz/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Przetwarzanie języka: af


Tłumaczenie tekstów: 4796wiersz [00:00, 7606.00wiersz/s]

Język bg nie jest obsługiwany
Język ca nie jest obsługiwany
Język cy nie jest obsługiwany
Język da nie jest obsługiwany
Język el nie jest obsługiwany
Język hu nie jest obsługiwany
Język kn nie jest obsługiwany
Język no nie jest obsługiwany
Przetwarzanie języka: pt


Tłumaczenie tekstów: 12661wiersz [05:37, 23.61wiersz/s] 

...

Język unknown nie jest obsługiwany
Przetwarzanie języka: ur


Tłumaczenie tekstów: 23745wiersz [28:51, 34.20wiersz/s]

Przetwarzanie języka: vi


Tłumaczenie tekstów: 25224wiersz [29:14, 14.38wiersz/s]

Język zh-cn nie jest obsługiwany
Język zh-tw nie jest obsługiwany
